<a href="https://colab.research.google.com/github/JGH94/Colab_ML_MovieReview/blob/main/MovieReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf  
import tensorflow_datasets as tfds  
import matplotlib.pyplot as plt

In [2]:
Dataset_, Info_ = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)  
Train_Dataset, Test_Dataset = Dataset_['train'], Dataset_['test']  
Encoder_ = Info_.features['text'].encoder 
print('Size : {}'.format(Encoder_.vocab_size)) 

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteMWUOHE/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteMWUOHE/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.
Size : 8185


In [3]:
Sample_String = "Hello WE-CO" 
Encode_String = Encoder_.encode(Sample_String) 
Original_String = Encoder_.decode(Encode_String)   
print(Encode_String) 
print(Original_String)  
for index in Encode_String: 
  print('{} ----> {}'.format(index, Encoder_.decode([index]))) 

[4025, 222, 6306, 7974, 2341]
Hello WE-CO
4025 ----> Hell
222 ----> o 
6306 ----> WE
7974 ----> -
2341 ----> CO


In [4]:
Buffer_Size = 10000 
Batch_Size = 64  
Train_Dataset = Train_Dataset.shuffle(Buffer_Size) 
Train_Dataset = Train_Dataset.padded_batch(Batch_Size) 
Test_Dataset = Test_Dataset.padded_batch(Batch_Size)

In [5]:
for x, y in Train_Dataset.take(1): 
  print(x) 
  print(y) 

tf.Tensor(
[[  19  175 4183 ...    0    0    0]
 [  19 2031   23 ...    0    0    0]
 [  19   32 6751 ...    0    0    0]
 ...
 [  12  239    7 ...    0    0    0]
 [ 298 1369   91 ...    0    0    0]
 [ 373    6    1 ...    0    0    0]], shape=(64, 1547), dtype=int64)
tf.Tensor(
[1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1
 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1], shape=(64,), dtype=int64)


In [6]:
model = tf.keras.Sequential([ 
    tf.keras.layers.Embedding(Encoder_.vocab_size, 64), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(1) 
    ])

In [7]:
Learning_Rate = 1e-4 
Epochs = 10 
Steps = 30 
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  
              optimizer=tf.keras.optimizers.Adam(Learning_Rate), 
              metrics=['accuracy']) 
history = model.fit(Train_Dataset, epochs=Epochs, validation_data=Test_Dataset, validation_steps= Steps)

Epoch 1/10
391/391 [==============================] - 131s 311ms/step - loss: 0.6392 - accuracy: 0.5702 - val_loss: 0.4464 - val_accuracy: 0.7818
Epoch 2/10
391/391 [==============================] - 120s 305ms/step - loss: 0.3469 - accuracy: 0.8480 - val_loss: 0.3460 - val_accuracy: 0.8609
Epoch 3/10
391/391 [==============================] - 120s 305ms/step - loss: 0.2497 - accuracy: 0.9035 - val_loss: 0.3416 - val_accuracy: 0.8698
Epoch 4/10
391/391 [==============================] - 121s 307ms/step - loss: 0.2099 - accuracy: 0.9228 - val_loss: 0.3318 - val_accuracy: 0.8719
Epoch 5/10
391/391 [==============================] - 119s 302ms/step - loss: 0.1849 - accuracy: 0.9313 - val_loss: 0.3499 - val_accuracy: 0.8714
Epoch 6/10
391/391 [==============================] - 119s 303ms/step - loss: 0.1638 - accuracy: 0.9420 - val_loss: 0.3700 - val_accuracy: 0.8677
Epoch 7/10
391/391 [==============================] - 120s 304ms/step - loss: 0.1490 - accuracy: 0.9476 - val_loss: 0.3731 -

In [8]:
def Sample_Predic(Sample_String): 
  encoded_sample_pred_text = Encoder_.encode(Sample_String)  
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32) 
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0)) 
  return(predictions)

In [9]:
#긍정리뷰
sample_pred_text = ('The movie was perfect') 
predictions = Sample_Predic(sample_pred_text) 
print(predictions)

[[0.7116385]]


In [12]:
#부정리뷰
sample_pred_text = ('this movie was very worst') 
predictions = Sample_Predic(sample_pred_text) 
print(predictions)

[[-0.9252117]]
